In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r"..")
from utils import *

In [2]:
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
train_gender = user['gender']
train_age = user['age']

In [5]:
from sklearn.model_selection import KFold
gender_oof = np.zeros((900000,2))
age_oof = np.zeros((900000,10))
gender_preds = []
age_preds = []

kfolder = KFold(n_splits=5, shuffle=True, random_state=2020)
kfold = kfolder.split(gender_oof)
fold_index = 1
for train_index, vali_index in kfold:      
    age_prob = np.load(f"{sub_path}/5fold_new/{fold_index}/age_prob.npy")
    gender_prob = np.load(f"{sub_path}/5fold_new/{fold_index}/gender_prob.npy", )
    age_val_prob = np.load(f"{sub_path}/5fold_new/{fold_index}/val_age_prob.npy", )
    gender_val_prob = np.load(f"{sub_path}/5fold_new/{fold_index}/val_gender_prob.npy", )

    fold_index += 1
    print(train_index, vali_index)

    gender_oof[vali_index] = gender_val_prob
    age_oof[vali_index] = age_val_prob

    gender_preds.append(gender_prob)
    age_preds.append(age_prob)
gender_prob = np.array(gender_preds).mean(axis=0)
age_prob = np.array(age_preds).mean(axis=0)


# gender_tune_weight = search_weight(train_gender[810000:900000]-1, gender_oof[810000:900000], init_weight=[1.0]*2,class_num=2, step=0.001)
# age_tune_weight = search_weight(train_age[810000:900000]-1, age_oof[810000:900000], init_weight=[1.0]*10,class_num=10, step=0.001)


# gender_prob_tune = np.array(gender_tune_weight)*gender_prob
gender_pre = np.argmax(gender_prob,axis=1) + 1

# age_prob_tune = np.array(age_tune_weight)*age_prob
age_pre = np.argmax(age_prob,axis=1) + 1



[     0      1      2 ... 899997 899998 899999] [     7     11     15 ... 899982 899988 899993]
[     0      1      2 ... 899996 899997 899999] [     3      4      9 ... 899980 899994 899998]
[     1      2      3 ... 899996 899998 899999] [     0      8     18 ... 899991 899992 899997]
[     0      2      3 ... 899994 899997 899998] [     1     12     14 ... 899995 899996 899999]
[     0      1      3 ... 899997 899998 899999] [     2      5      6 ... 899976 899981 899989]


In [6]:
sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_pre
sub['predicted_gender'] = gender_pre
print('ok！')
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')

ok！


In [5]:
age_prob = np.load(f"{sub_path}/age_prob.npy")
gender_prob = np.load(f"{sub_path}/gender_prob.npy")

age_val_prob = np.load(f"{sub_path}/val_age_prob.npy")
gender_val_prob = np.load(f"{sub_path}/val_gender_prob.npy")

age_tune_weight = search_weight(train_age[810000:900000]-1, age_val_prob, init_weight=[1.0]*10,class_num=10, step=0.001)
print(age_tune_weight)

gender_tune_weight = search_weight(train_gender[810000:900000]-1, gender_val_prob, init_weight=[1.0]*2,class_num=2, step=0.001)
print(gender_tune_weight)

gender_prob_tune = np.array(gender_tune_weight)*gender_prob
gender_pre = np.argmax(gender_prob_tune,axis=1) + 1

age_prob_tune = np.array(age_tune_weight)*age_prob
age_pre = np.argmax(age_prob_tune,axis=1) + 1

round:  1
0.47046666666666664
0.4705111111111111
0.47052222222222223
0.47055555555555556
0.4705888888888889
0.47063333333333335
0.4706444444444444
0.47067777777777775
0.4707222222222222
0.47075555555555554
0.4708111111111111
0.47092222222222224
0.4709777777777778
0.4710888888888889
0.47121111111111114
0.47124444444444447
0.47128888888888887
0.4713555555555556
0.4715111111111111
0.47152222222222223
0.47155555555555556
0.4718888888888889
0.47204444444444443
0.4721222222222222
0.47225555555555554
0.4724888888888889
0.4726666666666667
0.4729888888888889
0.473
0.47305555555555556
0.4731111111111111
0.47315555555555555
0.47336666666666666
0.47342222222222224
0.47352222222222223
0.47355555555555556
0.4737
0.4737222222222222
0.47383333333333333
0.47385555555555553
0.47386666666666666
0.4738777777777778
0.4739
0.47394444444444445
0.4739555555555556
0.4739888888888889
0.474
0.4740111111111111
0.47402222222222223
0.47404444444444443
0.47405555555555556
0.47407777777777776
round:  2
0.474133333333

## 融合LR

In [19]:
def fold5_mean(filename):
    gender_preds = []
    age_preds = []
    for i in [1,2,3,4,5]:
        age_prob = np.load(f"{sub_path}/{filename}/{i}/age_prob.npy")
        gender_prob = np.load(f"{sub_path}/{filename}/{i}/gender_prob.npy", )

        gender_preds.append(gender_prob)
        age_preds.append(age_prob)
    gender_prob = np.array(gender_preds).mean(axis=0)
    age_prob = np.array(age_preds).mean(axis=0)
    return gender_prob, age_prob
    
# 两个五折
gender_prob_51, age_prob_51 = fold5_mean('5fold')
gender_prob_52, age_prob_52 = fold5_mean('5fold_new')

# LR
age_prob_lr = np.load(f"{sub_path}/lr/lr_age_prob.npy")
gender_prob_lr = np.load(f"{sub_path}/lr/lr_gender_prob.npy")

# 单模

age_prob1 = np.load(f"{sub_path}/sub4157/age_prob.npy")
gender_prob1 = np.load(f"{sub_path}/sub4157/gender_prob.npy")

age_prob2 = np.load(f"{sub_path}/sub414/age_prob.npy")
gender_prob2 = np.load(f"{sub_path}/sub414/gender_prob.npy")

# age_prob3 = np.load(f"{sub_path}/sub413/age_prob.npy")
# gender_prob3 = np.load(f"{sub_path}/sub413/gender_prob.npy")

# age_prob4 = np.load(f"{sub_path}/sub4127/age_prob.npy")
# gender_prob4 = np.load(f"{sub_path}/sub4127/gender_prob.npy")

print(age_prob.shape, gender_prob.shape, age_prob_lr.shape, gender_prob_lr.shape)
print('------gender')
print(gender_prob_51.sum())
print(gender_prob_52.sum())
print(gender_prob_lr.sum())
print(gender_prob1.sum())
print(gender_prob2.sum())
# print(gender_prob3.sum())
# print(gender_prob4.sum())


print('------age')
print(age_prob_51.sum())
print(age_prob_52.sum())
print(age_prob_lr.sum())
print(age_prob1.sum())
print(age_prob2.sum())
# print(age_prob3.sum())
# print(age_prob4.sum())



sub_gender = np.argmax(gender_prob_51*0.35 + 
                       gender_prob_52*0.3 + 
                       gender_prob1*0.15 + 
                       gender_prob2*0.15 +  
                       gender_prob_lr*0.05 ,axis=1) + 1

sub_age = np.argmax(age_prob_51*0.35 + 
                       age_prob_52*0.3 + 
                       age_prob1*0.15 + 
                       age_prob2*0.15 +  
                       age_prob_lr*0.05 ,axis=1) + 1

# sub_gender = np.argmax((gender_prob_51 + 
#                        gender_prob_52 + 
#                        gender_prob1 + 
#                        gender_prob2)/4 ,axis=1) + 1

# sub_age = np.argmax((age_prob_51 + 
#                        age_prob_52 + 
#                        age_prob1 + 
#                        age_prob2 )/4 ,axis=1) + 1

print(sub_gender.shape, sub_age.shape)

sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = sub_age
sub['predicted_gender'] = sub_gender
print('ok！')
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')

(1000000, 10) (1000000, 2) (1000000, 10) (1000000, 2)
------gender
1000000.0
1000000.0
1000000.0
1000000.0
1000000.0
------age
999999.5
1000000.25
999999.9999999998
999999.8
999999.06
(1000000,) (1000000,)
ok！
